In [2]:
import atomica as at
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
loc3 = at.parent_dir()

In [4]:
loc3

WindowsPath('C:/Users/Phil/AppData/Local/Temp/ipykernel_33760')

## Load everything

In [2]:
P =at.Project(framework="hbv_v14_gamma_2.xlsx", databook="AFR_db_v1_2.xlsx", sim_start=1990, sim_end=2101, sim_dt=0.25, do_run=False) #test PSA on some parameters (vax eff, mtct_infx, mortality rates)

cal = P.make_parset()
cal.load_calibration('AFR_calib_v1_2.xlsx')

res = P.run_sim(parset=cal, result_name = 'Status Quo')

Elapsed time for running "default": 1.40s


In [27]:
res = P.run_sim(parset=cal, result_name = 'Central Results')

Elapsed time for running "default": 1.47s


In [4]:
len(res.get_variable('alive')[0].vals)

445

In [5]:
loc = 'C:/Users/Phil/Google Drive (phillip.luong@burnet.edu.au)/Projects/2022-12 VIMR HepB Modelling/2023-01_Wk2_Create_Demographic_db/Data/Templates'

In [6]:
np.random.seed(310123)
afr_ua=P.parsets[0]

n_samples = 10

parsets = {}
results = {}

for i in range(1, n_samples+1):
    parsets[i] = afr_ua.sample()
    
ages = ['0-4', '5-14', '15-49', '50-69', '70+']
trtinv = ['te_dc_cc', 'te_icl_ict', 'te_icl_cc', 'te_cc_dc', 'te_cc_hcc', 'te_ie_cc', 'te_m_dc', 'te_m_hcc', 'te_ict_hcc', 'te_ie_hcc', 'te_icl_hcc', 'te_dc_hcc']
vacinv = ['eag_ve', 'sag_ve', 'hb3_ve', 'mav_ve']
tot_pars = trtinv + vacinv

for i in range(1, n_samples+1):
    for age in ages:
        for par in tot_pars:
            parsets[i].get_par(par).ts[f'{age}F'].assumption = parsets[i].get_par(par).ts[f'{age}M'].assumption

## Input Results

In [8]:
ages = ['0-4', '5-14', '15-49', '50-69', '70+']
nathis = ['ci_p', 'm_acu', 'm_dc', 'm_hcc']
trtinv = ['te_dc_cc', 'te_icl_ict', 'te_icl_cc', 'te_cc_dc', 'te_cc_hcc', 'te_ie_cc', 'te_m_dc', 'te_m_hcc', 'te_ict_hcc', 'te_ie_hcc', 'te_icl_hcc', 'te_dc_hcc']
vacinv = ['eag_ve', 'sag_ve', 'hb3_ve', 'mav_ve']
tot_pars = trtinv + vacinv

In [9]:
in_df = pd.DataFrame(columns = ['run_id']+nathis+trtinv+vacinv)

In [25]:
for sim in range(10):
    in_df.loc[sim,'run_id'] = sim + 1
    for col in in_df.columns[1:]:
        in_df.loc[sim,col] = parsets[3].get_par(col).ts[0].assumption #TODO: get different input parameters per population

In [26]:
in_df.to_csv('input_results_v0.csv', index=False)

## Central Results

In [30]:
loc2 = 'Data/Demographics/'
df1 = pd.read_csv(loc2+'202212rfp-1_dds-202208_int_pop_both.csv')

df1 = df1[(df1.year >= 1990) & (df1.year <=2100)]
age_groups = ['0-4', '5-14', '15-49', '50-69', '70+']

for idx,row in df1.iterrows():
    if row.age_from < 5:
        df1.loc[idx,'age_group'] = '0-4'
    elif row.age_from < 15:
        df1.loc[idx,'age_group'] = '5-14'
    elif row.age_from < 50:
        df1.loc[idx,'age_group'] = '15-49'
    elif row.age_from < 70:
        df1.loc[idx,'age_group'] = '50-69'
    else:
        df1.loc[idx,'age_group'] = '70+'

In [31]:
output_dict = {'cohort_size': 'alive', 'cases': 'tot_inc', 'dalys':'dalys'}

In [33]:
cen_df = pd.read_csv(loc+'/central-burden-template.202212rfp-1.RFP_standard template_HepB.csv')

In [34]:
for opt in output_dict:
    tot_val = res.get_variable(output_dict[opt])[0].vals + res.get_variable(output_dict[opt])[1].vals
    for age in range(0,5):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val

    tot_val = res.get_variable(output_dict[opt])[2].vals + res.get_variable(output_dict[opt])[3].vals
    for age in range(5,15):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val

    tot_val = res.get_variable(output_dict[opt])[4].vals + res.get_variable(output_dict[opt])[5].vals
    for age in range(15,50):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val

    tot_val = res.get_variable(output_dict[opt])[6].vals + res.get_variable(output_dict[opt])[7].vals
    for age in range(50,70):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val

    tot_val = res.get_variable(output_dict[opt])[8].vals + res.get_variable(output_dict[opt])[9].vals
    for age in range(70,101):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val

In [35]:
tot_val = 0
for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
    tot_val += res.get_variable(var)[0].vals + res.get_variable(var)[1].vals
    
for age in range(0,5):
    for year in range(2000,2101):
        ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
        res_val = tot_val[(year-1990)*4]

        idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
        cen_df.loc[idx, 'deaths'] = ratio * res_val

tot_val = 0
for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
    tot_val += res.get_variable(var)[2].vals + res.get_variable(var)[3].vals
for age in range(5,15):
    for year in range(2000,2101):
        ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
        res_val = tot_val[(year-1990)*4]

        idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
        cen_df.loc[idx, 'deaths'] = ratio * res_val

tot_val = 0
for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
    tot_val += res.get_variable(var)[4].vals + res.get_variable(var)[5].vals
for age in range(15,50):
    for year in range(2000,2101):
        ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
        res_val = tot_val[(year-1990)*4]

        idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
        cen_df.loc[idx, 'deaths'] = ratio * res_val

tot_val = 0
for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
    tot_val += res.get_variable(var)[6].vals + res.get_variable(var)[7].vals
for age in range(50,70):
    for year in range(2000,2101):
        ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
        res_val = tot_val[(year-1990)*4]

        idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
        cen_df.loc[idx, 'deaths'] = ratio * res_val

tot_val = 0
for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
    tot_val += res.get_variable(var)[8].vals + res.get_variable(var)[9].vals
for age in range(70,101):
    for year in range(2000,2101):
        ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
        res_val = tot_val[(year-1990)*4]

        idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
        cen_df.loc[idx, 'deaths'] = ratio * res_val

## Stochastic Results

In [42]:
df = pd.read_csv(loc+'/stochastic-burden-template.202212rfp-1.RFP_standard template_HepB.csv') #template

In [39]:
for i in range(1, n_samples+1):
    results[i] = P.run_sim(parsets[i], result_name = f'sample result {i}')

Elapsed time for running "default": 1.39s
Elapsed time for running "default": 1.43s
Elapsed time for running "default": 1.49s
Elapsed time for running "default": 1.42s
Elapsed time for running "default": 1.44s
Elapsed time for running "default": 1.43s
Elapsed time for running "default": 1.43s
Elapsed time for running "default": 1.51s
Elapsed time for running "default": 1.42s
Elapsed time for running "default": 1.45s


In [43]:
dfs = {}
for sim in range(1,n_samples+1):
    stores = results[sim]
    print(f'Sim no {sim}')
    
    dfs[sim] = pd.read_csv(loc+'/stochastic-burden-template.202212rfp-1.RFP_standard template_HepB.csv')
    dfs[sim].run_id = sim+1
    ## Other outputs
    for opt in output_dict:
        print(opt)
        tot_val = stores.get_variable(output_dict[opt])[0].vals + stores.get_variable(output_dict[opt])[1].vals
        for age in range(0,5):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, opt] = ratio * res_val

        tot_val = stores.get_variable(output_dict[opt])[2].vals + stores.get_variable(output_dict[opt])[3].vals
        for age in range(5,15):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, opt] = ratio * res_val

        tot_val = stores.get_variable(output_dict[opt])[4].vals + stores.get_variable(output_dict[opt])[5].vals
        for age in range(15,50):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, opt] = ratio * res_val

        tot_val = stores.get_variable(output_dict[opt])[6].vals + stores.get_variable(output_dict[opt])[7].vals
        for age in range(50,70):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, opt] = ratio * res_val

        tot_val = stores.get_variable(output_dict[opt])[8].vals + stores.get_variable(output_dict[opt])[9].vals
        for age in range(70,101):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
                dfs[sim].loc[idx, opt] = ratio * res_val
                
    ## Deaths
    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += stores.get_variable(var)[0].vals + stores.get_variable(var)[1].vals

    for age in range(0,5):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
            dfs[sim].loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += stores.get_variable(var)[2].vals + stores.get_variable(var)[3].vals
    for age in range(5,15):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
            dfs[sim].loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += stores.get_variable(var)[4].vals + stores.get_variable(var)[5].vals
    for age in range(15,50):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
            dfs[sim].loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += stores.get_variable(var)[6].vals + stores.get_variable(var)[7].vals
    for age in range(50,70):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
            dfs[sim].loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += stores.get_variable(var)[8].vals + stores.get_variable(var)[9].vals
    for age in range(70,101):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = df[(dfs[sim].year == year) & (dfs[sim].age == age)].index[0]
            dfs[sim].loc[idx, 'deaths'] = ratio * res_val 

Sim no 2
cohort_size
cases
dalys
Sim no 3
cohort_size
cases
dalys
Sim no 4
cohort_size
cases
dalys
Sim no 5
cohort_size
cases
dalys
Sim no 6
cohort_size
cases
dalys
Sim no 7
cohort_size
cases
dalys
Sim no 8
cohort_size
cases
dalys
Sim no 9
cohort_size
cases
dalys
Sim no 10
cohort_size
cases
dalys
Sim no 11
cohort_size
cases
dalys


In [38]:
results

{}

In [ ]:
final_df = dfs[0]
for i in range(1,10):
    final_df = pd.concat([final_df,dfs[i]])

In [ ]:
final_df.to_csv('stochastic_results_v0.xlsx', index = False)